In [44]:
import pandas as pd

In [79]:
# Define a class called DataAnonymizer
class DataAnonymizer:
    
    # Initialize the class with a dataframe (df) and optionally, a list of sensitive columns
    def __init__(self, df, sensitive_columns=None):
        # Create a copy of the input dataframe and store it as an instance variable
        self.df = df.copy()
        
        # Check the type of sensitive_columns and store it as an instance variable
        if sensitive_columns is None:
            print('Sensitive Column is empty')  # Print a message if sensitive_columns is None
            self.sensitive_columns = []  # Initialize an empty list
        elif isinstance(sensitive_columns, str):
            self.sensitive_columns = [sensitive_columns]  # Convert a single string to a list with one item
        else:
            self.sensitive_columns = sensitive_columns  # Store the provided list
        display(df)
        
    # Method to redact values in the dataframe that are less than a minimum threshold but not zero
    def less_than_threshold_not_zero(self, frequency, minimum_threshold=10):
        # Create a boolean mask that identifies rows where the column specified by 'frequency'
        # has values less than 'minimum_threshold' and not equal to zero
        mask = (self.df[frequency] < minimum_threshold) & (self.df[frequency] != 0)

        self.df['RedactBinary'] = 0

        self.df.loc[mask, 'RedactBinary'] = 1
        
        # Update a new column named 'Redact' with a message for the rows that meet the condition specified by the mask
        self.df.loc[mask, 'Redact'] = f'Less Than {minimum_threshold} and not zero'
        
        # Return the updated dataframe
        return self.df

    # Method to redact values in the dataframe that are overlapping with other redacted values
    def redact_threshold(self, frequency, organization_columns=None, minimum_threshold=10):
        # Call another method to handle rows where the frequency is less than 'minimum_threshold' but not zero
        self.less_than_threshold_not_zero(frequency, minimum_threshold)
        
        # Initialize 'Overlapping' column to zero
        self.df['Overlapping'] = 0
        
        # Loop through each sensitive column to check for overlapping sensitive information
        for sensitive_column in self.sensitive_columns:
            list_sensitive = self.df[~self.df['Redact'].isnull()][sensitive_column].unique()
            self.df.loc[self.df[sensitive_column].isin(list_sensitive), 'Overlapping'] += 1
        
        # Mark rows with maximum overlapping as 'Suppressed'
        self.df.loc[self.df['Overlapping'] == self.df['Overlapping'].max(), 'Suppressed'] = 1
        
        # Update 'Redact' column for rows where 'Overlapping' is maximum but 'Redact' is null
        mask = ((self.df['Overlapping'] == self.df['Overlapping'].max()) & (self.df['Redact'].isnull()))
        self.df.loc[mask, 'Redact'] = 'Overlapping threshold secondary suppression'
        
        # Finalize the columns to be retained in the dataframe
        if organization_columns is None:
            print('No organization columns')
            self.df = self.df[self.sensitive_columns + [frequency] + ['Redact']]
        elif isinstance(organization_columns, str):
            organization_columns = [organization_columns]
            self.df = self.df[organization_columns + self.sensitive_columns + [frequency] + ['Redact']]
        else:
            self.df = self.df[organization_columns + self.sensitive_columns + [frequency] + ['Redact']]
            
        # Return the modified dataframe
        return self.df

    # Method to redact values in the dataframe that are the sum of minimum threshold 
    def sum_redact(self, frequency, organization_columns=None, minimum_threshold=10):
        # Filter rows where the value in column specified by 'frequency' is less than 'minimum_threshold' but not zero
        df_less_than = self.df[(self.df[frequency] < minimum_threshold) & (self.df[frequency] != 0)]
        
        print(frequency)
        
        # Group the filtered dataframe by 'organization_columns' and sum the values in 'frequency'
        df_grouped_less_than = df_less_than.groupby(organization_columns)[frequency].sum().reset_index()
        
        df_grouped_less_than.rename(columns={frequency: "TotalCount"}, inplace=True)
        
        # Further filter the grouped dataframe to retain only rows where 'TotalCount' is less or equal to 'minimum_threshold'
        df_filtered_group = df_grouped_less_than[df_grouped_less_than['TotalCount'] <= minimum_threshold]
        
        # Select only the 'organization_columns' from the filtered grouped dataframe
        df_filtered = df_filtered_group[organization_columns]
        
        # Filter rows where the value in the column specified by 'frequency' is greater than or equal to 'minimum_threshold'
        df_greater_than = self.df[self.df[frequency] >= minimum_threshold]
        
        # Group the filtered dataframe by 'organization_columns' and get the minimum value in the 'frequency' column
        df_grouped_greater_than = df_greater_than.groupby(organization_columns)[frequency].min().reset_index()
        
        df_grouped_greater_than.rename(columns = {frequency: "MinimumValue"}, inplace=True)
        
        # Merge the filtered and grouped dataframes based on 'organization_columns'
        df_result = pd.merge(df_filtered, df_grouped_greater_than, on=organization_columns, how='inner')
        
        # List columns for the dataframe with values greater than or equal to 'minimum_threshold'
        greater_than_columns = organization_columns + ['MinimumValue']
        
        # List columns for merging the original dataframe with the result dataframe
        merged_columns = organization_columns + [frequency]
        
        # Merge the original dataframe with the result dataframe based on 'merged_columns' and 'greater_than_columns'
        self.df = pd.merge(self.df, df_result, left_on=merged_columns, right_on=greater_than_columns, how='left')
        
        self.df.loc[~self.df['MinimumValue'].isnull(), 'RedactBinary'] = 1
        
        # Update the 'Redact' column with a specific message for rows where 'MinimumValue' is not null
        self.df.loc[self.df['RedactBinary'] == 1, 'Redact'] = 'Sum of minimum threshold redact needed secondary suppression'
        
        # Retain only the necessary columns in the final dataframe
        self.df = self.df[organization_columns + self.sensitive_columns + [frequency] + ['Redact', 'RedactBinary']]
        
        # Return the updated dataframe
        return self.df

    # Method to redact values in the dataframe that are the only value in the group
    def one_count_redacted(self, frequency, organization_columns=None, minimum_threshold=10):
        # Filter rows where the value in the column specified by 'frequency' is less than 'minimum_threshold' but not zero
        df_less_than_eleven_count = self.df[(self.df[frequency] < minimum_threshold) & (self.df[frequency] != 0)]
        
        # Group the filtered dataframe by 'organization_columns' and count the size of each group
        df_grouped = df_less_than_eleven_count.groupby(organization_columns).count().reset_index()
        
        df_grouped.rename(columns={frequency: "counts"}, inplace=True)
        
        # Further filter the grouped dataframe to retain only rows where 'counts' equals 1
        df_filtered_grouped = df_grouped[df_grouped['counts'] == 1]
        
        # Select only the 'organization_columns' from the filtered grouped dataframe
        df_filtered = df_filtered_grouped[organization_columns]
        
        # Filter rows where the value in the column specified by 'frequency' is greater than or equal to 'minimum_threshold'
        df_minimum_threshold = self.df[self.df[frequency] >= minimum_threshold]
        
        # Group the filtered dataframe by 'organization_columns' and get the minimum value in the 'frequency' column
        df_grouped_min = df_minimum_threshold.groupby(organization_columns)[frequency].min().reset_index()
        
        df_grouped_min.rename(columns = {frequency: "MinimumValue"}, inplace=True)
        
        # Merge the filtered and grouped dataframes based on 'organization_columns'
        df_result = pd.merge(df_filtered, df_grouped_min, on=organization_columns, how='inner')
        
        # List columns for the dataframe with values greater than or equal to 'minimum_threshold'
        greater_than_columns = organization_columns + ['MinimumValue']
        
        # List columns for merging the original dataframe with the result dataframe
        merged_columns = organization_columns + [frequency]
        
        # Merge the original dataframe with the result dataframe based on 'merged_columns' and 'greater_than_columns'
        self.df = pd.merge(self.df, df_result, left_on=merged_columns, right_on=greater_than_columns, how='left')

        self.df.loc[~self.df['MinimumValue'].isnull(), 'RedactBinary'] = 1
        
        # Update the 'Redact' column with a specific message for rows where 'MinimumValue' is not null
        self.df.loc[self.df['RedactBinary'] == 1, 'Redact'] = 'Count minimum threshold needed secondary suppression'
        
        # Retain only the necessary columns in the final dataframe
        self.df = self.df[organization_columns + self.sensitive_columns + [frequency] + ['Redact', 'RedactBinary']]
        
        # Return the updated dataframe
        return self.df
    
    def one_redact_zero (self, frequency, organization_colummns):        
        # Filtering the DataFrame based on School Year and SuppressionID        
        df_filtered = self.df[~self.df['Redact'].isnull()]    
        
        # Grouping by DimSeaID and counting StudentCount, then filtering groups with a single record  
        df_grouped_count = df_filtered.groupby(organization_colummns).count().reset_index()  
        
        df_grouped_count.rename(columns={frequency: "counts"}, inplace=True)
        
        df_filtered_grouped_count = df_grouped_count[df_grouped_count['counts'] == 1]
        
        df_filtered_grouped_count['Zero'] = 1          
        
        # Merge the original DataFrame with the filtered grouped DataFrame based on DimSeaID        
        self.df = pd.merge(self.df, df_filtered_grouped_count, on=organization_colummns, how='left')                
        self.df.loc[(self.df[frequency] == 0) & self.df['Zero'] == 1, 'Redact'] = 'Redact zero needed for secondary suppression'
        
        return self.df
        
    def organization_group_redaction(self, frequency, organization_columns=None, minimum_threshold=10):
        return self.df

# Sample DataFrame

In [64]:
df_data = pd.read_csv("SampleDataELLGender.csv")
print(list(df_data))
#display(df_data)
df_grouped = df_data.groupby(['ChildEntity', 'Subgroup1'])['GraduationCount'].sum().reset_index()

anonymizer = DataAnonymizer(df_grouped, sensitive_columns=['Subgroup1'])
anonymizer.less_than_threshold_not_zero('GraduationCount')
anonymizer.sum_redact('GraduationCount', ['ChildEntity'])
df_grouped_suppressed = anonymizer.one_count_redacted('GraduationCount', ['ChildEntity'])
df_grouped_suppressed.loc[df_grouped_suppressed['RedactBinary'] == 1, 'RedactedGroup'] = 1
df_data = df_data.merge(df_grouped_suppressed, on=['ChildEntity', 'Subgroup1', 'GraduationCount'], how='left')
display(df_data)
# display(df_data.groupby(['ChildEntity', 'Subgroup2'])['GraduationCount'].sum())
# display(df_data.groupby(['ParentEntity', 'Subgroup1'])['GraduationCount'].sum())
# display(df_data.groupby(['ParentEntity', 'Subgroup2'])['GraduationCount'].sum())

['ParentEntity', 'ChildEntity', 'Subgroup1', 'Subgroup2', 'GraduationCount', 'CohortCount', 'GraduationRate']


,ChildEntity,Subgroup1,GraduationCount
0,School1,English Language Learner,44
1,School1,Not English Language Learner,49
2,School2,English Language Learner,55
3,School2,Not English Language Learner,27
4,School3,English Language Learner,46
5,School3,Not English Language Learner,25
6,School4,English Language Learner,133
7,School4,Not English Language Learner,54
8,School5,English Language Learner,15
9,School5,Not English Language Learner,9


TypeError: reset_index() got an unexpected keyword argument 'names'

In [65]:
df_data = pd.read_csv("DataDegreeBeth.csv")
display(df_data)



# Instantiate the anonymizer
anonymizer = DataAnonymizer(df_data, sensitive_columns=['Subgroup1', 'Subgroup2'])
# Apply Less Than Value Not Zero
df_redact_threshold = anonymizer.redact_threshold('Counts')

display(df_redact_threshold)

,Subgroup1,Subgroup2,Counts,UnderSuppress,OverSuppressed
0,STEM,Certificate,10,0,1
1,STEM,Associate,9,1,1
2,STEM,Bachelor,20,0,0
3,STEM,Masters,100,1,0
4,STEM,Doctorate,40,0,0
5,Business,Certificate,15,0,1
6,Business,Associate,40,0,0
7,Business,Bachelor,15,0,1
8,Business,Masters,90,0,0
9,Business,Doctorate,11,0,1


,Subgroup1,Subgroup2,Counts,UnderSuppress,OverSuppressed
0,STEM,Certificate,10,0,1
1,STEM,Associate,9,1,1
2,STEM,Bachelor,20,0,0
3,STEM,Masters,100,1,0
4,STEM,Doctorate,40,0,0
5,Business,Certificate,15,0,1
6,Business,Associate,40,0,0
7,Business,Bachelor,15,0,1
8,Business,Masters,90,0,0
9,Business,Doctorate,11,0,1


No organization columns


,Subgroup1,Subgroup2,Counts,Redact
0,STEM,Certificate,10,NaN
1,STEM,Associate,9,Less Than 10 and not zero
2,STEM,Bachelor,20,NaN
3,STEM,Masters,100,Overlapping threshold secondary suppression
4,STEM,Doctorate,40,NaN
5,Business,Certificate,15,NaN
6,Business,Associate,40,NaN
7,Business,Bachelor,15,NaN
8,Business,Masters,90,NaN
9,Business,Doctorate,11,NaN


In [80]:
# Sample DataFrame with 'Organization', 'Parent_Organization', and 'Gender'
data = {
    'Parent_Organization': ['Parent1', 'Parent1', 'Parent1', 'Parent1', 'Parent1', 'Parent1', 'Parent2', 'Parent2', 'Parent2', 'Parent2', 'Parent2', 'Parent2'],
    'Organization': ['Org1', 'Org1', 'Org1', 'Org2', 'Org2', 'Org2', 'Org3', 'Org3', 'Org3', 'Org4', 'Org4', 'Org4'],
    'Gender': ['F', 'M', 'NB', 'F', 'M', 'NB', 'F', 'M', 'NB', 'F', 'M', 'NB'],
    'GraduationCount': [20, 0, 2, 13, 5, 4, 25, 13, 7, 10, 0, 3]
}

df = pd.DataFrame(data)

df = pd.read_csv('SampleData.csv')

# Instantiate the anonymizer
anonymizer = DataAnonymizer(df, ['Subgroup'])

# Apply Less Than Value Not Zero
anonymizer.less_than_threshold_not_zero('GraduationCount')

# Sum of Suppressed Values
anonymizer.sum_redact('GraduationCount', ['ParentEntity', 'ChildEntity'] )

# One Count of Suppressed Values
anonymizer.one_count_redacted('GraduationCount', ['ParentEntity', 'ChildEntity'])

# Apply Redaction on Zero Values
anonymizer.one_redact_zero('GraduationCount', ['ParentEntity', 'ChildEntity'])

display(anonymizer.df)

,ParentEntity,ChildEntity,Subgroup,GraduationCount,CohortCount,GraduationRate
0,District1,School1,Male,14,18,0.777778
1,District1,School1,Female,7,44,0.159091
2,District1,School1,Non-binary,22,33,0.666667
3,District1,School2,Male,5,13,0.384615
4,District1,School2,Female,4,9,0.444444
5,District1,School2,Non-binary,34,45,0.755556
6,District2,School3,Male,13,13,1.000000
7,District2,School3,Female,4,100,0.040000
8,District2,School3,Non-binary,8,92,0.086957
9,District2,School4,Male,43,44,0.977273


GraduationCount


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:175: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,ParentEntity,ChildEntity,Subgroup_x,GraduationCount,Redact_x,RedactBinary_x,Subgroup_y,counts,Redact_y,RedactBinary_y,Zero,Redact
0,District1,School1,Male,14,Count minimum threshold needed secondary suppr...,1,NaN,NaN,NaN,NaN,NaN,NaN
1,District1,School1,Female,7,Count minimum threshold needed secondary suppr...,1,NaN,NaN,NaN,NaN,NaN,NaN
2,District1,School1,Non-binary,22,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
3,District1,School2,Male,5,Count minimum threshold needed secondary suppr...,1,NaN,NaN,NaN,NaN,NaN,NaN
4,District1,School2,Female,4,Count minimum threshold needed secondary suppr...,1,NaN,NaN,NaN,NaN,NaN,NaN
5,District1,School2,Non-binary,34,Count minimum threshold needed secondary suppr...,1,NaN,NaN,NaN,NaN,NaN,NaN
6,District2,School3,Male,13,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
7,District2,School3,Female,4,Count minimum threshold needed secondary suppr...,1,NaN,NaN,NaN,NaN,NaN,NaN
8,District2,School3,Non-binary,8,Count minimum threshold needed secondary suppr...,1,NaN,NaN,NaN,NaN,NaN,NaN
9,District2,School4,Male,43,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
